In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os

In [12]:
json_file_path = os.path.join("./annotations", "bapproach1.json")
with open(json_file_path, 'r') as f:
    single_json_data = json.load(f)

single_json_data
categories = single_json_data["categories"]
annotations = single_json_data["annotations"]
categories
annotations
categories[annotations[0]["category_id"]-1]["name"]

'approach'

In [13]:
import os
import json
import pandas as pd

json_folder = "./annotations"
csv_folder = "./output"
output_folder = "./labeled_output"

os.makedirs(output_folder, exist_ok=True)

for json_file in os.listdir(json_folder):
    if json_file.endswith(".json"):
        json_path = os.path.join(json_folder, json_file)
        csv_path = os.path.join(csv_folder, json_file.replace(".json", ".csv"))
        
        if not os.path.exists(csv_path):
            print(f"El archivo CSV para {json_file} no existe. Saltando...")
            continue
        
        with open(json_path, 'r') as f:
            json_data = json.load(f)

        categories = json_data["categories"]
        
        annotated_frames = {ann["image_id"] for ann in json_data["annotations"]}
        annotated_frames_categories = {ann["category_id"] for ann in json_data["annotations"]}

        csv_data = pd.read_csv(csv_path)

        csv_data["annotation"] = csv_data["frame"].apply(
            lambda frame: "Still" if frame not in annotated_frames else categories[json_data["annotations"][list(annotated_frames)[0]]["category_id"]-1]["name"]
        )
        
        output_csv_path = os.path.join(output_folder, os.path.basename(csv_path))
        csv_data.to_csv(output_csv_path, index=False)
        print(f"Procesado: {json_file} -> {output_csv_path}")

print("Procesamiento completado.")

Procesado: bapproach1.json -> ./labeled_output\bapproach1.csv
Procesado: bapproach2.json -> ./labeled_output\bapproach2.csv
Procesado: bapproach3.json -> ./labeled_output\bapproach3.csv
Procesado: bback1.json -> ./labeled_output\bback1.csv
Procesado: bback2.json -> ./labeled_output\bback2.csv
Procesado: bback3.json -> ./labeled_output\bback3.csv
Procesado: bjump1.json -> ./labeled_output\bjump1.csv
Procesado: bjump2.json -> ./labeled_output\bjump2.csv
Procesado: bjump3.json -> ./labeled_output\bjump3.csv
Procesado: blturn1.json -> ./labeled_output\blturn1.csv
Procesado: blturn2.json -> ./labeled_output\blturn2.csv
Procesado: blturn3.json -> ./labeled_output\blturn3.csv
Procesado: blwalk1.json -> ./labeled_output\blwalk1.csv
Procesado: blwalk2.json -> ./labeled_output\blwalk2.csv
Procesado: brturn1.json -> ./labeled_output\brturn1.csv
Procesado: brturn2.json -> ./labeled_output\brturn2.csv
Procesado: brturn3.json -> ./labeled_output\brturn3.csv
Procesado: brwalk1.json -> ./labeled_outpu

In [16]:
import os
import pandas as pd

# Folder containing the CSV files
folder_path = 'labeled_output'

# List to store processed DataFrames
dataframes = []

# Iterate over all CSV files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        videoid = os.path.splitext(file_name)[0]  # Extract the videoid from the file name
        
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Extract the annotation for the frame
        df_annotation = df[['frame', 'annotation']].drop_duplicates(subset=['frame'])
        
        # Pivot the rest of the data
        pivot_df = df.pivot(index='frame', columns='landmark', values=['x', 'y', 'z'])
        
        # Flatten the multi-index columns
        pivot_df.columns = [f"{col[1]}_{col[0]}" for col in pivot_df.columns]
        
        # Add the annotation and videoid columns
        pivot_df = pivot_df.merge(df_annotation, on='frame')
        pivot_df['videoid'] = videoid
        
        # Append to the list
        dataframes.append(pivot_df)

# Concatenate all DataFrames
final_df = pd.concat(dataframes, ignore_index=True)

# Save the final DataFrame to a CSV file
final_output_path = 'merged_output.csv'
final_df.to_csv(final_output_path, index=False)

print(f"Data merged and saved to {final_output_path}")


Data merged and saved to merged_output.csv


In [18]:
import numpy as np
import pandas as pd

def calculate_angle(p1, p2, p3):
    """
    Calculate the angle at p2 formed by the line segments p1-p2 and p3-p2.
    Arguments:
    - p1, p2, p3: Arrays or lists containing the coordinates [x, y, z].
    Returns:
    - Angle in degrees.
    """
    # Create vectors
    v1 = np.array(p1) - np.array(p2)
    v2 = np.array(p3) - np.array(p2)

    # Calculate the dot product and magnitudes
    dot_product = np.dot(v1, v2)
    magnitude_v1 = np.linalg.norm(v1)
    magnitude_v2 = np.linalg.norm(v2)

    # Avoid division by zero
    if magnitude_v1 == 0 or magnitude_v2 == 0:
        return None

    # Calculate the angle in radians
    angle_radians = np.arccos(dot_product / (magnitude_v1 * magnitude_v2))
    # Convert to degrees
    angle_degrees = np.degrees(angle_radians)
    return angle_degrees

# Load the CSV
input_file = "merged_output.csv"  # Replace with your input file
output_file = "output_with_angles.csv"

# Columns for the joints
columns = {
    "LEFT_HIP": ["LEFT_HIP_x", "LEFT_HIP_y", "LEFT_HIP_z"],
    "LEFT_KNEE": ["LEFT_KNEE_x", "LEFT_KNEE_y", "LEFT_KNEE_z"],
    "LEFT_ANKLE": ["LEFT_ANKLE_x", "LEFT_ANKLE_y", "LEFT_ANKLE_z"],
    "RIGHT_HIP": ["RIGHT_HIP_x", "RIGHT_HIP_y", "RIGHT_HIP_z"],
    "RIGHT_KNEE": ["RIGHT_KNEE_x", "RIGHT_KNEE_y", "RIGHT_KNEE_z"],
    "RIGHT_ANKLE": ["RIGHT_ANKLE_x", "RIGHT_ANKLE_y", "RIGHT_ANKLE_z"],
}

# Read the data
df = pd.read_csv(input_file)

# Calculate angles for each frame
angles = []
for _, row in df.iterrows():
    left_knee_angle = calculate_angle(
        row[columns["LEFT_HIP"]], 
        row[columns["LEFT_KNEE"]], 
        row[columns["LEFT_ANKLE"]]
    )
    right_knee_angle = calculate_angle(
        row[columns["RIGHT_HIP"]], 
        row[columns["RIGHT_KNEE"]], 
        row[columns["RIGHT_ANKLE"]]
    )
    angles.append({"LEFT_KNEE_ANGLE": left_knee_angle, "RIGHT_KNEE_ANGLE": right_knee_angle})

# Add angles to the DataFrame
angles_df = pd.DataFrame(angles)
result_df = pd.concat([df, angles_df], axis=1)

# Save to a new CSV
result_df.to_csv(output_file, index=False)
print(f"Processed data saved to {output_file}")


Processed data saved to output_with_angles.csv


In [19]:
movement_data = pd.read_csv("output_with_angles.csv")
movement_data

,frame,LEFT_ANKLE_x,LEFT_EAR_x,LEFT_ELBOW_x,LEFT_EYE_x,LEFT_EYE_INNER_x,LEFT_EYE_OUTER_x,LEFT_FOOT_INDEX_x,LEFT_HEEL_x,LEFT_HIP_x,...,RIGHT_INDEX_z,RIGHT_KNEE_z,RIGHT_PINKY_z,RIGHT_SHOULDER_z,RIGHT_THUMB_z,RIGHT_WRIST_z,annotation,videoid,LEFT_KNEE_ANGLE,RIGHT_KNEE_ANGLE
0,0,0.584068,0.549615,0.609942,0.540257,0.535122,0.545035,0.605685,0.577719,0.563867,...,-0.465094,0.025656,-0.419580,-0.265862,-0.389978,-0.367329,Still,bapproach1,143.113804,130.907198
1,1,0.583816,0.549585,0.609758,0.540175,0.535058,0.544640,0.604018,0.577247,0.563768,...,-0.446831,0.026957,-0.401221,-0.255758,-0.374700,-0.352086,Still,bapproach1,137.527439,129.889167
2,2,0.583469,0.549162,0.609155,0.539909,0.534654,0.543987,0.603049,0.576701,0.562722,...,-0.447805,0.026783,-0.403685,-0.259966,-0.375523,-0.353428,Still,bapproach1,135.966635,129.388538
3,3,0.583131,0.547480,0.608366,0.539257,0.533717,0.542982,0.602520,0.576462,0.561531,...,-0.473416,0.031153,-0.427835,-0.273019,-0.399217,-0.376418,Still,bapproach1,136.770951,131.594040
4,4,0.582929,0.545416,0.606326,0.537320,0.531471,0.541365,0.602478,0.576202,0.559685,...,-0.475887,0.037990,-0.430928,-0.272486,-0.402016,-0.379456,Still,bapproach1,137.860208,135.599254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7582,75,0.540843,0.509875,0.575166,0.500046,0.495905,0.503950,0.561585,0.535347,0.533014,...,-0.369331,0.094562,-0.342578,-0.224297,-0.318219,-0.304534,stand,sstand2,145.458334,152.490991
7583,76,0.540866,0.509719,0.572911,0.499869,0.495740,0.503922,0.561563,0.535014,0.531667,...,-0.332582,0.109827,-0.305114,-0.190400,-0.281306,-0.267339,stand,sstand2,148.419924,155.225842
7584,77,0.540867,0.508466,0.571057,0.499365,0.495548,0.503570,0.561559,0.534524,0.530355,...,-0.317961,0.088072,-0.289987,-0.169468,-0.266304,-0.252134,stand,sstand2,144.655286,150.169795
7585,78,0.540918,0.506948,0.568187,0.498815,0.495290,0.502902,0.561532,0.534506,0.529315,...,-0.306402,0.082846,-0.278941,-0.159719,-0.255165,-0.241358,stand,sstand2,142.435515,149.059785
